In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src')

In [2]:
import torch
import torchvision

# Choose hardware acceleration if available
def choose_device() -> str:
    if torch.cuda.is_available():
        return "cuda:0"
    if hasattr(torch.backends, "mps"):
        if torch.backends.mps.is_available():
            return "mps"
    return "cpu"


device = torch.device(choose_device())
device

/home/aletheia/.virtualenvs/torchxai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=0)

In [3]:
# Adjust this path.
path_to_files = "./assets/imagenet"

# Load test data and make loaders.
x_batch = torch.load(f'{path_to_files}/x_batch.pt')
y_batch = torch.load(f'{path_to_files}/y_batch.pt')
s_batch = torch.load(f'{path_to_files}/s_batch.pt')
x_batch, s_batch, y_batch = x_batch.to(device), s_batch.to(device), y_batch.to(device)
print(f"{len(x_batch)} matches found.")

17 matches found.


In [4]:
# Load pre-trained ResNet18 model.
model = torchvision.models.resnet18(pretrained=True)
model = model.to(device)

/home/aletheia/.virtualenvs/torchxai/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/aletheia/.virtualenvs/torchxai/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
from captum.attr import IntegratedGradients

# Generate Integrated Gradients attributions of the first batch of the test set.
ig = IntegratedGradients(model)
a_batch, conv_delta = ig.attribute(x_batch, target=y_batch, baselines=torch.zeros_like(x_batch), internal_batch_size=1, return_convergence_delta=True)

/home/aletheia/.virtualenvs/torchxai/lib/python3.10/site-packages/captum/attr/_utils/batching.py:45: UserWarning: Internal batch size cannot be less than the number of input examples. Defaulting to internal batch size of 17 equal to the number of examples.
  warnings.warn(


In [6]:
from torchxai.metrics import completeness
completeness_scores = completeness(model, x_batch, a_batch, baselines=torch.zeros_like(x_batch), target=y_batch)

from torchxai.metrics import monotonicity_corr_and_non_sens
monotonicity_corr, non_sens_scores = monotonicity_corr_and_non_sens(model, x_batch, a_batch, target=y_batch, max_features_processed_per_example=64)

print(f"Completeness: {completeness_scores}")
print(f"Monotonicity correlation: {monotonicity_corr}")
print(f"Non-sensitivity scores: {non_sens_scores}")

/home/aletheia/.virtualenvs/torchxai/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


KeyboardInterrupt: 